In [13]:
import os

In [14]:
os.getcwd()

'e:\\Personal Files\\Learning Programming\\Data Science\\Projects\\ChestCancerPrediction\\research'

In [15]:
os.chdir("../")

In [16]:
os.getcwd()

'e:\\Personal Files\\Learning Programming\\Data Science\\Projects\\ChestCancerPrediction'

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [4]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from ChestCancerPrediction.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from ChestCancerPrediction.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=Path(config.unzip_dir)
        )
        
        return data_ingestion_config

In [7]:
import os
import sys
import zipfile
import gdown
from ChestCancerPrediction import logger
from ChestCancerPrediction.exception import CustomeException
from ChestCancerPrediction.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self) -> str:
        '''
        Fetch data from the google drive
        '''
        try:
            dataset_url = self.config.source_URL
            zip_download_url = self.config.local_data_file
            os.makedirs(self.config.root_dir,exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_url}")
            
            file_id = dataset_url.split("/")[-2]
            download_link = f"https://drive.google.com/uc?id={file_id}&confirm=t"
            gdown.download(download_link,zip_download_url,quiet=False)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_url}")
            
        except Exception as e:
            raise CustomeException(e,sys)
        
    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        '''
        try:
            unzip_path = self.config.unzip_dir
            with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
                zip_ref.extractall(unzip_path)
        
        except Exception as e:
            raise CustomeException(e,sys)

In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise CustomeException(e,sys)

[2024-12-09 12:18:14,145: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-09 12:18:14,147: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-09 12:18:14,149: INFO: common: created directory at: artifacts]
[2024-12-09 12:18:14,150: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-09 12:18:14,152: INFO: 2759775983: Downloading data from https://drive.google.com/file/d/1NiTRDuShDvwA4GEAinLXCdjMS3pJGy4k/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?id=1NiTRDuShDvwA4GEAinLXCdjMS3pJGy4k&confirm=t
To: e:\Personal Files\Learning Programming\Data Science\Projects\ChestCancerPrediction\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:01<00:00, 38.5MB/s]

[2024-12-09 12:18:17,428: INFO: 2759775983: Downloaded data from https://drive.google.com/file/d/1NiTRDuShDvwA4GEAinLXCdjMS3pJGy4k/view?usp=sharing into file artifacts/data_ingestion/data.zip]
